# COMP9318 Lab4

## Instructions
1. This note book contains instructions for **COMP9318-Lab4**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures via corresponding functions.

* You can submit your implementation for **Lab4** via following link: https://unswkg.net/submit/ .

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* You are allowed to add other functions and/or import modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 20th April, 2020 (SYDNEY TIME)**. We will **not** accept any late submissions.

# Question-1: Text Classification using Multinomial Naive Bayes

You are required to implement a multinomial naive bayes classifier to predict spam SMS.

The training data is a set of SMS categoried into `spam` and `ham`.

In [28]:
import pandas as pd

raw_data = pd.read_csv('./asset/data.txt', sep='\t')
raw_data

,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...


In order to implement a unigram model, first we tokenize the text. We use the count corresponding to each token (word) in the SMS as its feature (i.e., bag of words). We store the features and catrgorical information for each SMS in a `dictionary`.

In [124]:
def tokenize(sms):
    return sms.split(' ')

def get_freq_of_tokens(sms):
    tokens = {}
    for token in tokenize(sms):
        if token not in tokens:
            tokens[token] = 1
        else:
            tokens[token] += 1
    return tokens

training_data = []
for index in range(len(raw_data)):
    training_data.append((get_freq_of_tokens(raw_data.iloc[index].text), raw_data.iloc[index].category))

In [126]:
tokenize(sms)

['I', 'am', 'not', 'spam']

For this lab, you need to **implement** a multinomial naive bayes classifier (i.e., `multinomial_nb()` in the file: `submission.py`) with add-1 smoothing. The input arguments of `multinomial_nb()` are:
* `training_data`: pre-processed data stored as a `dictionary`
* `sms`: test-sms (i.e., a list of tokens) that you need to categorize as `spam` and/or `ham`

The return value of `multinomial_nb()` should be the **ratio** of the probability of sms is spam and the probability of sms is ham. A return value larger than 1 implies the `sms` is spam and vice versa.

For example, a sample output is shown in the cell given below:

In [3]:
## How we test your implementation...
import submission_ans as submission

sms = 'I am not spam'
print(submission.multinomial_nb(training_data, tokenize(sms)))

0.2342767295597484




# Test Environment

For testing, we have pre-installed the requisite modules and/or libraries in the testing environment. You are only allowed to use following libraries:
* python: 3.6.5
* pandas: 0.19.2

NOTE: You are required to implement the classifier by yourself. You are not allowed to import **sklearn** and/or any other library in Lab4.

In [270]:
sms = ['I', 'am', 'not', 'spam']
def multinomial_nb(training_data, sms):
    
    # prevent x not in training_data
    smooth = 1
    voc = set()
    num = {'ham': 0, 'spam': 0}
    prior = {'ham': 0, 'spam': 0}
    sentence = {'spam':{},'ham':{}}
    
    for data in training_data:
        prior[data[1]] += 1
        for index in data[0]:
            num[data[1]] += data[0][index]
            voc.add(index)
            if index in sentence[data[1]]:
                sentence[data[1]][index] += data[0][index]
            else:
                sentence[data[1]][index] = data[0][index]
    # probability of 'ham'
    prob0 = prior['ham'] / len(training_data)
    # probability of 'spam'
    prob1 = prior['spam'] / len(training_data)
    result = {'ham': prob0, 'spam': prob1}
    

    for token in sms:
        if token in voc:
            for key in sentence:
                if token in sentence[key]:
                    conditional_prob = (sentence[key][token] + smooth) / (num[key] + len(voc))
                else:
                    conditional_prob = ( smooth) / (num[key] + len(voc))
                result[key] *= conditional_prob
    return result['spam'] / result['ham']

    
                    

In [271]:
multinomial_nb(training_data, sms)

0.23427672955974846

In [235]:
def train(training_data):
    smooth = 1
    vocabulary = set()
    data_value = {'spam':0,'ham':0}
    data_set= {'spam':[],'ham':[]}
    prior_prob = {'spam':0,'ham':0}
    conditional_prob = dict()


    for data in training_data:
        prior_prob[data[1]] += 1
        data_value[data[1]] += sum(data[0].values())
        for i in data[0].items():
            data_set[data[1]].append(i)
            vocabulary.add(i[0])
    # print(len(vocabulary))
    prior_prob['spam'] /=  len(training_data)
    prior_prob['ham'] /= len(training_data)
    # print(prior_prob)
#     print(data_value)
#     print(data_set)

    print(len(voc))
    for data in training_data:
        for i in data[0]:
            print(i)
            for j in ('spam', 'ham'):
                print(j)
                t = 0
                for x in data_set[j]:
                    if x[0] == i:
                        t += x[1]
                print(t)
                temp = data_value[j] + len(vocabulary)
                print(temp)
                conditional_prob[(i, j)] = (sum([x[1] for x in data_set[j] if x[0] == i]) + smooth )/ ((data_value[j] + len(vocabulary)))
    print(data_value)
    return vocabulary, conditional_prob, prior_prob

def predict(sms ,vocabulary, conditional_prob, prior_prob):
    result ={'spam': prior_prob['spam'], 'ham': prior_prob['ham']}
    for token in sms:
        if token in vocabulary:
            print(f'The token is {token}')
            for i in ['spam', 'ham']:
                # print((token,i),':' ,conditional_prob[(token,i)])
                print(f'The result[{i}] is {result[i]}')
                print(f'The {(token, i)} is {conditional_prob[(token, i)]}')
                result[i] *= conditional_prob[(token, i)]
                print(f'The result[{i}] is {result[i]}')
    # print(result['spam'])
    # print(result['ham'])
    return result['spam']/ result['ham']